In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4o-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4o-mini")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def compute_agent_cost(agent):
    input_token_price = 0.15 / 1000000
    output_token_price = 0.6 / 1000000
    return (
        input_token_price * agent.monitor.get_total_token_counts()["input"]
        + output_token_price * agent.monitor.get_total_token_counts()["output"]
    )

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [5]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [6]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can provide information, answer questions, and assist with problem-solving across a variety of topics.

Cost of the agent:  0.0001485


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [7]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I am unable to browse the internet or visit external websites. However, you can check the recent news directly on the Swissinfo website at https://www.swissinfo.ch/eng/.

Cost of the agent:  0.0001623


In [8]:
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [9]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using  tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else yo

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [10]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [11]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

Recent news from Swissinfo:
1. "Can the WTO live up to its mission?" - Discusses the challenges faced by the WTO amidst rising trade tensions.
2. "What is a tariff? A quick guide" - Explains the concept of tariffs and their implications.
3. "Swiss business lobby warns Trump tariff turmoil could escalate" - A look into the concerns raised by Swiss businesses over tariffs.
4. "Switzerland’s global image got a boost in 2024" - An analysis of Switzerland's improved global image this year.
5. "Explainer: Why US health funding cuts are rattling Swiss science" - Discusses how US health funding cuts are affecting Swiss scientific research.
6. "Swiss move on with self-driving buses" - Reports on Switzerland's advancements in autonomous vehicle technology.
7. "How Switzerland’s political system of direct democracy works" - An educational piece on Switzerland's unique political system.
8. "The rocky relationship between Switzerland and the European Union" - A discussion on the complex dynamics between Switzerland and the EU.

Cost of the agent:  0.0008889


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [12]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using  tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else yo

In [13]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using  tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answe

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [14]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [15]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
    verbosity_level=0,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [16]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

7

Cost of the agent:  0.00033465000000000003


In [17]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using  tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else yo

### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [18]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [19]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [20]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 1.78 seconds| Input tokens: 996 | Output tokens: 22]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 0.53 seconds| Input tokens: 2,090 | Output tokens: 36]

7

Cost of the agent:  0.00033509999999999996


## Code Agent

In [21]:
from smolagents import DuckDuckGoSearchTool

web_search = DuckDuckGoSearchTool()
agent = ToolCallingAgent(
    model=model,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
    planning_interval=5,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [22]:
output = agent.run(
    "Give me the growth of population in Switzerland between 2020 and 2000", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the growth of population in Switzerland between 2020 and 2000                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Search for the population of Switzerland in the year 2000 using a web search to find reliable sources.
2. Search for the population of Switzerland in the year 2020 using a web search to find reliable sources.
3. Calculate the total population growth from 2000 to 2020 by finding the difference between the two population 
figures obtained.
4. Calculate the average annual population growth rate by dividing the total growth figure by 20 (the number of 
years).
5. Compile the results and prepare the final answer addressing the population growth in Switzerland between the 
years 2000 and 2020.
6. Provide the final answer using the final_answer tool.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'Switzerland population in 2000'}                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: ## Search Results

|Switzerland Population 1950-2025 - 
Macrotrends](https://www.macrotrends.net/global-metrics/countries/CHE/switzerland/population)
Chart and table of Switzerland population from 1950 to 2025. United Nations projections are also included through 
the year 2100. The current population of Switzerland in 2025 is 8,904,610, a 0.6% increase from 2024.; The 
population of Switzerland in 2024 was 8,851,431, a 0.62% increase from 2023.; The population of Switzerland in 2023
was 8,796,669, a 0.64% increase from 2022.

|Demographics of Switzerland - Wikipedia](https://en.wikipedia.org/wiki/Demographics_of_Switzerland)
The population of Switzerland 1970-2005. Data from Swiss Federal Statistical Office 2005; number of inhabitants in 
thousands. Population pyramid of Switzerland in 2021 by citizenship. ... In 2000, 5.78 million residents (79.2%, 
compared to 93.7% in 1980) were Christian ...

|Switzerland - Population 2000 - 
countryeconomy.com](https://countryeconomy.com/demography/population/switzerland?year=2000)
The female population is greater, with 3,684,357 women, representing 51.14% of the total, compared to 3,519,698 or 
48.86% men. Switzerland, with 174 people per square km and it was in position 155th in our ranking of density 
population in 2000. In this page we show you the progression of the population in Switzerland.

|Population of Switzerland 2000 - PopulationPyramid.net](https://www.populationpyramid.net/switzerland/2000/)
Population Pyramids: Switzerland - 2000. Mailing List - See more. PopulationPyramid.net Population Pyramids of the 
World from 1950 to 2100. Switzerland. AFRICA ASIA Afghanistan Albania Algeria Angola Antigua and Barbuda Argentina 
Armenia Aruba Australia Australia/New Zealand Austria Azerbaijan Bahamas Bahrain Bangladesh Barbados Belarus 
Belgium ...

|Population of Switzerland 1800-2020 - 
Statista](https://www.statista.com/statistics/1067098/population-switzerland-historical/)
Today, with a population of 8.7 million, Switzerland is ranked among the wealthiest and most developed nations in 
the world, with very high standards of living. Read more

|Federal Population Census (1850-2000) - Bundesamt für 
Statistik](https://www.bfs.admin.ch/bfs/en/home/statistics/population/surveys/vz.html)
Federal Population Census (1850-2000) - Bundesamt für Statistik

|Switzerland - Population, Migration, Diversity | 
Britannica](https://www.britannica.com/place/Switzerland/Demographic-trends)
Switzerland - Population, Migration, Diversity: Since the beginning of the 20th century, the population of 
Switzerland has more than doubled, from about 3.3 million in 1900 to more than 8 million in the early 21st century.
... In 2000 Swiss voters ratified the new agreement, but it was not until 2016 that Switzerland officially withdrew
its ...

|Switzerland Population 2025 - World Population Review](https://worldpopulationreview.com/countries/switzerland)
Population of Switzerland. Switzerland's population structure shows a nearly equal male to female ratio of 0.99 to 
1, ... The nationwide census of 2000 revealed that there were 7,452,075 people living here and a subsequent 
estimate in 2011 suggested that the population of Switzerland had increased to 7,952,600, ...

|Population growth in Switzerland - 
Worlddata.info](https://www.worlddata.info/europe/switzerland/populationgrowth.php)
2000: 7.18 M: 0.56 %: ... Birth and death rate in Switzerland Population growth is the result of the birth rate, 
the mortality rate and the migration rate. For example, the year 2021 in the graph: The population in Switzerland 
increased by about 66,400 inhabitants. In the same year, the death rate was 8.2 per 1,000 people (~ 70,800 deaths) 
and ...

|Census in Switzerland - Wikipedia](https://en.wikipedia.org/wiki/Census_in_Switzerland)
The first Federal population census in Switzerland took place in March 1850 under the direction of Federal 
Councillor Stefano Franscini. As well as counting the number of inha

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The population of Switzerland in 2000 was              │
│ approximately 7,450,000. The population in 2020 was approximately 8,667,100. The growth of the population       │
│ between 2000 and 2020 is about 1,217,100.'}                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The population of Switzerland in 2000 was approximately 7,450,000. The population in 2020 was 
approximately 8,667,100. The growth of the population between 2000 and 2020 is about 1,217,100.

[Step 3: Duration 1.53 seconds| Input tokens: 7,827 | Output tokens: 193]

The population of Switzerland in 2000 was approximately 7,450,000. The population in 2020 was approximately 8,667,100. The growth of the population between 2000 and 2020 is about 1,217,100.

Cost of the agent:  0.00128985


In [23]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
    planning_interval=5,
)
agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [24]:
output = agent.run(
    "Give me the growth of population in Switzerland between 2020 and 2000", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the growth of population in Switzerland between 2020 and 2000                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Perform a web search to find the population of Switzerland in the year 2000.
2. Perform a web search to find the population of Switzerland in the year 2020.
3. Calculate the absolute population growth by subtracting the population in 2000 from the population in 2020 using
the sum_tool.
4. Calculate the percentage growth by taking the absolute change, dividing it by the population in 2000, and then 
multiplying by 100.
5. Provide the final answer that includes both the absolute growth and the percentage growth.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  population_2000 = web_search(query="Switzerland population in 2000")                                             
  print("Population of Switzerland in 2000:", population_2000)                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Population of Switzerland in 2000: ## Search Results

[Switzerland Population 1950-2025 - 
Macrotrends](https://www.macrotrends.net/global-metrics/countries/CHE/switzerland/population)
Chart and table of Switzerland population from 1950 to 2025. United Nations projections are also included through 
the year 2100. The current population of Switzerland in 2025 is 8,904,610, a 0.6% increase from 2024.; The 
population of Switzerland in 2024 was 8,851,431, a 0.62% increase from 2023.; The population of Switzerland in 2023
was 8,796,669, a 0.64% increase from 2022.

[Demographics of Switzerland - Wikipedia](https://en.wikipedia.org/wiki/Demographics_of_Switzerland)
The population of Switzerland 1970-2005. Data from Swiss Federal Statistical Office 2005; number of inhabitants in 
thousands. Population pyramid of Switzerland in 2021 by citizenship. ... In 2000, 5.78 million residents (79.2%, 
compared to 93.7% in 1980) were Christian ...

[Switzerland - Population 2000 - 
countryeconomy.com](https://countryeconomy.com/demography/population/switzerland?year=2000)
The female population is greater, with 3,684,357 women, representing 51.14% of the total, compared to 3,519,698 or 
48.86% men. Switzerland, with 174 people per square km and it was in position 155th in our ranking of density 
population in 2000. In this page we show you the progression of the population in Switzerland.

[Population of Switzerland 2000 - PopulationPyramid.net](https://www.populationpyramid.net/switzerland/2000/)
Population Pyramids: Switzerland - 2000. Mailing List - See more. PopulationPyramid.net Population Pyramids of the 
World from 1950 to 2100. Switzerland. AFRICA ASIA Afghanistan Albania Algeria Angola Antigua and Barbuda Argentina 
Armenia Aruba Australia Australia/New Zealand Austria Azerbaijan Bahamas Bahrain Bangladesh Barbados Belarus 
Belgium ...

[Population of Switzerland 1800-2020 - 
Statista](https://www.statista.com/statistics/1067098/population-switzerland-historical/)
Today, with a population of 8.7 million, Switzerland is ranked among the wealthiest and most developed nations in 
the world, with very high standards of living. Read more

[Federal Population Census (1850-2000) - Bundesamt für 
Statistik](https://www.bfs.admin.ch/bfs/en/home/statistics/population/surveys/vz.html)
Federal Population Census (1850-2000) - Bundesamt für Statistik

[Switzerland Population 2025 - World Population Review](https://worldpopulationreview.com/countries/switzerland)
Population of Switzerland. Switzerland's population structure shows a nearly equal male to female ratio of 0.99 to 
1, ... The nationwide census of 2000 revealed that there were 7,452,075 people living here and a subsequent 
estimate in 2011 suggested that the population of Switzerland had increased to 7,952,600, ...

[Population growth in Switzerland - 
Worlddata.info](https://www.worlddata.info/europe/switzerland/populationgrowth.php)
2000: 7.18 M: 0.56 %: ... Birth and death rate in Switzerland Population growth is the result of the birth rate, 
the mortality rate and the migration rate. For example, the year 2021 in the graph: The population in Switzerland 
increased by about 66,400 inhabitants. In the same year, the death rate was 8.2 per 1,000 people (~ 70,800 deaths) 
and ...

[Switzerland - Federal Population Census 2000 - IPUMS 
Subset](https://microdata.worldbank.org/index.php/catalog/2128)
Switzerland, Federal Statistical Office, Federal Population Census 2000 The licensing agreement for use of 
IPUMS-International data requires that users supply IPUMS-International with the title and full citation for any 
publications, research reports, or educational materials making use of the data or documentation.

[Switzerland Population - JetPunk](https://www.jetpunk.com/data/population/countries/switzerland)
For example, the population of Germany in 1970 includes both East Germany and West Germany. Years after 2023 are 
projections based on the UN's "medium fertility" scen

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "absolute_growth": absolute_growth,                                                                          
      "percentage_growth": percentage_growth                                                                       
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'absolute_growth': 1215025, 'percentage_growth': 16.304519211092213}

[Step 4: Duration 1.05 seconds| Input tokens: 15,934 | Output tokens: 479]

Cost of the agent:  0.0026775


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [25]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to run web search to obtain information. Call it as a 
        │   function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
            │              │ your query (think a Google search) then   │ perform.                                 │
            │              │ returns the top search results.           │                                          │
            │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
            │              │ problem.                                  │ problem                                  │
            └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘

In [26]:
output = manager_agent.run(
    "Give me the growth of population in Switzerland between 2020 and 2000", reset=True
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(manager_agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me the growth of population in Switzerland between 2020 and 2000                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  population_2000 = information_retriever_agent(task="Find the population of Switzerland in the year 2000.")       
  population_2020 = information_retriever_agent(task="Find the population of Switzerland in the year 2020.")       
  print("Population in 2000:", population_2000)                                                                    
  print("Population in 2020:", population_2020)                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - information_retriever_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'information_retriever_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Find the population of Switzerland in the year 2000.                                                            │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(1466245)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 1466245

[Step 3: Duration 1.31 seconds| Input tokens: 9,322 | Output tokens: 536]

Cost of the agent:  0.0017199


In [ ]:
agent.write_memory_to_messages()

## Visualize Your Agent

In [ ]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="Manager Agent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [30]:
import requests

class PDFExtractorAPI:
    def __init__(self):
        pass

    def extract_text_and_images(self, pdf_file_path: str) -> tuple[str, list]:
        BASE_URL = "https://compute.datascience.ch/vlm2vec-innovation"
        # Send the request with the file directly
        with open(pdf_file_path, "rb") as pdf_file:
            files = {"pdf_file": (pdf_file_path, pdf_file, "application/pdf")}
            response = requests.post(
                f"{BASE_URL}/v1/pdf_to_markdown",
                files=files
            )

        # Process the response
        result = response.json()
        markdown_text = result["markdown"]
        images = result["images"]

        return result, markdown_text, images


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractorAPI().extract_text_and_images(pdf_file_path)
    return text

In [31]:
tmp = read_pdf_tool("../data/hypoxy_stat_1page.pdf")
tmp

'Please cite this article in press as: Blume et al., HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity, Cell (2025), https://doi.org/10.1016/j.cell.2025.01.029\n\n![](_page_0_Picture_2.jpeg)\n\n### Article HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity\n\nSkyler Y. Blume,[1,](#page-0-0)[2](#page-0-1) Ankur Garg,[1,](#page-0-0)[2](#page-0-1) Yolanda Martı´-Mateos,[1,](#page-0-0)[2](#page-0-1) Ayush D. Midha,[1,](#page-0-0)[2](#page-0-1) Brandon T. L. Chew,[1,](#page-0-0)[2](#page-0-1) Baiwei Lin,[3](#page-0-2) Cecile Yu,[3](#page-0-2) Ryan Dick,[3](#page-0-2) Patrick S. Lee,[3](#page-0-2) Eva Situ,[3](#page-0-2) Richa Sarwaikar,[3](#page-0-2) Eric Green,[3](#page-0-2) Vyas Ramanan,[3](#page-0-2) Gijsbert Grotenbreg,[3](#page-0-2) Maarten Hoek,[3](#page-0-2) Christopher Sinz,[3](#page-0-2) and Isha H. Jain[1,](#page-0-0)[2,](#page-0-1)[4,](#page-0-3)[5,](#page-0-4)[\\\\*](#page-0-5)\n\n<span id="

In [32]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── read_pdf_agent | CodeAgent | gpt-4o-mini
    │   ├── ✅ Authorized imports: []
    │   ├── 📝 Description: Reads and summarize a PDF file. Call it as a function using the **task** argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ read_pdf_tool │ This tool reads a PDF file and returns   │ pdf_file_path (`string`): The path to    │
    │       │               │ the text content of the PDF file.        │ the PDF file.                            │
    │       │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │       │               │ on your query (think a Google search)    │ perform.                                 │
    │       │               │ then returns the top search results.     │                                          │
    │       │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │       │               │ reads its content as a markdown string.  │ to visit.                                │
    │       │               │ Use this to browse webpages.             │                                          │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── web_search_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you. Call it as a function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
            │               │ on your query (think a Google search)    │ perform.                                 │
            │               │ then returns the top search results.     │                                          │
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  

In [33]:
output = agent.run(
    "Can you read the PDF file at '../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────── New run - medical_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you read the PDF file at '../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give   │
│ me the wikipedia definition of the area of research?                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  read_pdf_agent(task="Please read the PDF file at '../data/hypoxy_stat_1page.pdf' and summarize its content.")    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - read_pdf_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'read_pdf_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Please read the PDF file at '../data/hypoxy_stat_1page.pdf' and summarize its content.                          │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "pdf_summary": "The PDF discusses HypoxyStat, a small-molecule therapy that enhances oxygen-hemoglobin       
  affinity, showing promise in treating mitochondrial diseases by inducing systemic hypoxia. This has resulted in  
  extended lifespan and improved health in animal models. The therapy allows for oral dosing and produces          
  systemic hypoxia even under normoxic conditions.",                                                               
      "wikipedia_definition": "Hypoxia is characterized by insufficient oxygen supply. Hypoxia therapy,            
  especially intermittent hypoxic therapy (IHT), involves controlled low oxygen exposure to improve physical       
  functions, with uses in sports medicine and rehabilitation."                                                     
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'pdf_summary': 'The PDF discusses HypoxyStat, a small-molecule therapy that enhances 
oxygen-hemoglobin affinity, showing promise in treating mitochondrial diseases by inducing systemic hypoxia. This 
has resulted in extended lifespan and improved health in animal models. The therapy allows for oral dosing and 
produces systemic hypoxia even under normoxic conditions.', 'wikipedia_definition': 'Hypoxia is characterized by 
insufficient oxygen supply. Hypoxia therapy, especially intermittent hypoxic therapy (IHT), involves controlled low
oxygen exposure to improve physical functions, with uses in sports medicine and rehabilitation.'}

[Step 5: Duration 6.50 seconds| Input tokens: 23,144 | Output tokens: 2,098]

Cost of the agent:  0.0047304


## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [34]:
from src.data_classes import Chunk
from src.data_processing import SimpleChunker, PDFExtractorAPI
from src.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from src.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [35]:
data_folder = "../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [36]:
data_extractor = PDFExtractorAPI()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [37]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [38]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [39]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_documents(chunks, embeddings)

In [40]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

[[{'chunk_id': '48',
   'score': 1.2692097425460815,
   'chunk': Chunk(chunk_id=48, content='performed the research. F.A. and N.T. analyzed the data. F.A., N.T., and R.C. contributed new reagents/analytical tools. ### **FUNDING INFORMATION** This study was sponsored by the healthcare business of Merck KGaA, Darmstadt, Germany (CrossRef Funder ID:10.13039/100009945). Editorial support was provided by Syneos Health, London, UK, and funded by the healthcare business of Merck KGaA, Darmstadt, Germany. ### **CONFLICT OF INTEREST STATEMENT** N.T. is an employee of Ares Trading S.A., Lausanne, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. R.S., R.B. and A.J. are employees of the healthcare business of Merck KGaA, Darmstadt, Germany. K.B. was an employee of the healthcare business of Merck KGaA, Darmstadt, Germany at the time of the study. V.G. is an employee of Ares Trading S.A. Eysins, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. F.A. and R.C.', metadata={'sour

### Toolify

In [44]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.
    
    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """
    
    return retriever.retrieve(query, number_of_chunks)

# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It returns a list of string │ chunks to return, by default it's 5.        │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [46]:
# Call the agent
output = rag_agent.run("According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = retriever_tool(query="SHAP analysis factors predicting higher-grade edema (Grade 2+)")          
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'search_results = retriever_tool(query="SHAP analysis factors predicting higher-grade
edema (Grade 2+)")' due to: TypeError: retriever_tool() missing 1 required positional argument: 'number_of_chunks'

[Step 1: Duration 2.05 seconds| Input tokens: 2,080 | Output tokens: 87]

...

[Step 2: Duration 2.56 seconds| Input tokens: 4,404 | Output tokens: 195]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "factors": {                                                                                                 
          "past_current_edema_grade": "History of Grade 2+ edema increases likelihood of future occurrences.",     
          "albumin_levels": "Lower albumin levels are linked to higher risk of Grade 2+ edema.",                   
          "age": "Age greater than 70 years increases likelihood of Grade 2+ edema.",                              
          "cumulative_dose": "Lower recent doses correlate with reduced probability of Grade 2+ edema."            
      }                                                                                                            
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'factors': {'past_current_edema_grade': 'History of Grade 2+ edema increases likelihood of 
future occurrences.', 'albumin_levels': 'Lower albumin levels are linked to higher risk of Grade 2+ edema.', 'age':
'Age greater than 70 years increases likelihood of Grade 2+ edema.', 'cumulative_dose': 'Lower recent doses 
correlate with reduced probability of Grade 2+ edema.'}}

[Step 3: Duration 2.79 seconds| Input tokens: 8,265 | Output tokens: 467]

In [45]:
output = rag_agent.run("What is the highest court of the USA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the highest court of the USA?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Supreme Court")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Supreme Court

[Step 1: Duration 2.58 seconds| Input tokens: 2,066 | Output tokens: 64]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents


In [69]:
#
from smolagents import DuckDuckGoSearchTool

web_search_agent = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

rag_agent = CodeAgent(name="medical_literature_agent",
                    description="Retrieve information from medical litterature. Call it as a function using the **task** argument.", 
                      tools=[retriever_tool], 
                      model=model)

agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[rag_agent, web_search_agent],
    verbosity_level=1,
)

In [70]:
# Call the agent
output = agent.run("According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  task = "Retrieve information about SHAP analysis factors that predict higher-grade edema (Grade 2+)."            
  medical_literature_agent(task=task)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────── New run - medical_literature_agent ───────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'medical_literature_agent'.                                                        │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Retrieve information about SHAP analysis factors that predict higher-grade edema (Grade 2+).                    │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 10.41 seconds| Input tokens: 4,034 | Output tokens: 574]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Factors such as past edema grade, albumin levels, age >70, and medication dosage predict           
  higher-grade edema (Grade 2+).")                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Factors such as past edema grade, albumin levels, age >70, and medication dosage predict 
higher-grade edema (Grade 2+).

[Step 2: Duration 1.68 seconds| Input tokens: 6,742 | Output tokens: 759]

In [71]:
# Call the agent
output = agent.run("Who is the current prime minister of Canada as of 2025?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Who is the current prime minister of Canada as of 2025?                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_pm_search = web_search(task="Who is the current prime minister of Canada as of 2025?")                   
  print(current_pm_search)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'current_pm_search = web_search(task="Who is the current prime minister of Canada as 
of 2025?")' due to: InterpreterError: It is not permitted to evaluate other functions than the provided tools or 
functions defined/imported in previous code (tried to execute web_search).

[Step 1: Duration 1.18 seconds| Input tokens: 2,089 | Output tokens: 102]

...

[Step 2: Duration 5.09 seconds| Input tokens: 5,424 | Output tokens: 401]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Mark Carney is the current Prime Minister of Canada as of 2025.")                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Mark Carney is the current Prime Minister of Canada as of 2025.

[Step 3: Duration 0.91 seconds| Input tokens: 8,160 | Output tokens: 477]